<a href="https://colab.research.google.com/github/Gregory-Eales/RL-TicTacToe/blob/master/RL_TicTacToe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dependencies

In [0]:
!pip3 install dropbox

    100% |████████████████████████████████| 512kB 22.8MB/s 


In [0]:
import numpy as np
import keras
import dropbox

Creating Tic Tac Toe Engine

In [0]:
class TicTacToeENV(object):
  
  def __init__(self):
    self.board = np.zeros([3, 3])
    self.dbx_token = "qCE45upb8QAAAAAAAAAAEkIQWUdP5SqMqWG9I5C_3DDAN_Vlleeyjvc5Sv4vyWp9"
    self.dbx = dropbox.Dropbox(self.dbx_token)
    
  
  ###########################
  # GAME ENVIROMENT METHODS #
  ###########################
  
  # runs the main game loop
  def play(self):
    
    # reset board
    self.board = np.zeros([3, 3])
    # get player move
    
    # check if move is valid
    
    # if valid change turn
    
    pass
  
  
  # checks to see if move is valid
  def check_move(self):
    pass
  
  # gets human move
  def get_move(self):
    pass
  
  # check to see if there is a winner
  def check_winner(self):
    pass
  
  
  ####################
  # DROP BOX METHODS #
  ####################
  
  # returns the current dropbox account
  def get_dbx_account(self):
    return(self.dbx.users_get_current_account())
  
  # uploads project file
  def upload_file_dbx(self):
    pass
  
  # downloads project file
  def download_file_dbx(self):

Testing DropBox

In [0]:
env = TicTacToeENV()
env.get_dbx_account()

FullAccount(account_id='dbid:AABwjUNGNrFbraMKLoAMQOgQAyyAsdOy-Mc', name=Name(given_name='Gregory', surname='Eales', familiar_name='Gregory', display_name='Gregory Eales', abbreviated_name='GE'), email='gregoeales@aol.com', email_verified=True, disabled=False, locale='en', referral_link='https://db.tt/YFb0K0gYnb', is_paired=False, account_type=AccountType('basic', None), root_info=UserRootInfo(root_namespace_id='2383773264', home_namespace_id='2383773264'), profile_photo_url=None, country='US', team=None, team_member_id=None)